# Notebook to analyze data

In [67]:
import pandas as pd
import numpy as np

In [78]:
stats = pd.read_csv("/home/ludop/Documents/asr-assisted-transcription/data/output/stats.csv", sep="\t")
stats["module"] = [x.split("_")[1][:-1] for x in stats["Transcript_ID"]]
stats

,Transcript_ID,num_speakers,transcribed_minutes,num_total_tokens,num_linguistic_tokens,num_metaling_tokens,num_shortpauses,num_unknown,num_errors,num_overlaps,...,ling_tokens_per_min::2,ling_tokens_per_min::3,ling_tokens_per_min::4,ling_tokens_per_min::5,ling_tokens_per_min::6,ling_tokens_per_min::7,ling_tokens_per_min::8,ling_tokens_per_min::9,ling_tokens_per_min::10,module
0,03_StraParlaC_whi,3,4.301600,718,694,6,15,3,0,106,...,512,694,0,0,0,0,0,0,0,StraParla
1,01_StraParlaA_I,2,5.426967,930,876,5,42,6,1,132,...,521,712,876,0,0,0,0,0,0,StraParla
2,02_StraParlaA_V,2,4.237667,789,757,4,25,2,1,50,...,523,712,757,0,0,0,0,0,0,StraParla
3,02_StraParlaA_A,2,4.038000,792,756,1,34,1,0,104,...,570,756,0,0,0,0,0,0,0,StraParla
4,01_ParlaBOA_M,2,3.339950,602,586,7,6,3,0,52,...,506,586,0,0,0,0,0,0,0,ParlaBO
5,03_ParlaBOA,2,8.282717,1448,1433,12,0,3,0,128,...,508,658,832,1044,1206,1361,1433,0,0,ParlaBO
6,03_StraParlaB_whi,2,6.938883,1223,1199,5,17,2,0,154,...,529,709,881,1036,1199,0,0,0,0,StraParla
7,01_StraParlaA_L,2,2.796717,522,512,2,8,0,0,52,...,512,0,0,0,0,0,0,0,0,StraParla
8,03_ParlaBOB,2,9.859083,1719,1686,27,0,4,2,144,...,467,605,791,975,1159,1333,1507,1669,1686,ParlaBO
9,01_PastiB_V,3,3.213167,634,597,7,18,10,2,50,...,586,597,0,0,0,0,0,0,0,Pasti


## numero di token trascritti



### Isoliamo solo i token linguistici (togliamo shortpauses e token metalinguistici)


In [ ]:
# TUTTI I TOKEN
# TODO: forse bisogna sostituire gli 0 con N/A nel dataframe?
# columns =
avg_per_minute = stats[["module", "transcription_type"]+[f"tokens_per_minute::{i}" for i in range(0,11)]].replace(0, np.nan).groupby(["module", "transcription_type"]).mean()
# avg_per_minute.loc[("ParlaBO", "Revised"), "tokens_per_minute::8"]

avg_per_minute_ling = stats[["module", "transcription_type"]+[f"ling_tokens_per_min::{i}" for i in range(0,11)]].replace(0, np.nan).groupby(["module", "transcription_type"]).mean()
# avg_per_minute_ling

| ParlaBO | Delta(FS) | Delta(whi) |

In [87]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 11):
            diff = avg_per_minute.loc[(module, t1), f"tokens_per_minute::{i}"] - avg_per_minute.loc[(module, t2), f"tokens_per_minute::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,ParlaBO,Revised - From-Scratch,-5.500000,-7.500000,-16.333333,46.500000,-13.500000,-3.0,-16.0,-3.5,50.0,NaN,NaN
1,ParlaBO,Revised - Whisper-Assisted,1.500000,-0.500000,-8.000000,-3.500000,66.500000,99.0,97.0,94.5,160.0,NaN,NaN
2,Pasti,Revised - From-Scratch,5.666667,15.333333,28.666667,148.333333,43.500000,274.0,NaN,NaN,NaN,NaN,NaN
3,Pasti,Revised - Whisper-Assisted,4.333333,21.666667,40.666667,42.333333,47.166667,45.0,30.0,36.0,NaN,NaN,NaN
4,StraParla,Revised - From-Scratch,0.900000,8.000000,23.300000,75.083333,-62.800000,NaN,NaN,NaN,NaN,NaN,NaN
5,StraParla,Revised - Whisper-Assisted,-18.850000,-8.650000,-21.100000,0.000000,33.200000,22.0,19.0,NaN,NaN,NaN,NaN


In [89]:
delta_pairs = [
    ("Revised", "From-Scratch"),
    ("Revised", "Whisper-Assisted"),
    # Add more pairs as needed
]
modules = ["ParlaBO", "Pasti", "StraParla"]

# Create an empty list to store rows
rows = []

# Iterate over module names
for module in modules:
    for t1, t2 in delta_pairs:
        row = [module, f"{t1} - {t2}"]
        for i in range(0, 11):
            diff = avg_per_minute_ling.loc[(module, t1), f"ling_tokens_per_min::{i}"] - avg_per_minute_ling.loc[(module, t2), f"ling_tokens_per_min::{i}"]
            row.append(diff)
        rows.append(row)

# Create a new DataFrame
columns = avg_per_minute.columns # Column names
# print(columns)
df_deltas = pd.DataFrame(rows, columns=["module", "delta"].extend(columns))
df_deltas

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,ParlaBO,Revised - From-Scratch,-3.000000,-2.500000,-10.500000,51.500000,-9.500000,1.5,-13.5,-3.0,50.0,NaN,NaN
1,ParlaBO,Revised - Whisper-Assisted,3.000000,6.166667,2.833333,11.166667,79.500000,97.5,94.5,93.0,158.0,NaN,NaN
2,Pasti,Revised - From-Scratch,6.833333,19.166667,34.666667,153.500000,42.500000,260.0,NaN,NaN,NaN,NaN,NaN
3,Pasti,Revised - Whisper-Assisted,5.166667,22.500000,39.000000,42.833333,48.166667,39.0,21.0,26.0,NaN,NaN,NaN
4,StraParla,Revised - From-Scratch,0.000000,4.600000,18.700000,72.000000,-43.600000,NaN,NaN,NaN,NaN,NaN,NaN
5,StraParla,Revised - Whisper-Assisted,-19.800000,-12.500000,-26.250000,-7.000000,21.400000,15.0,10.0,NaN,NaN,NaN,NaN


2. il numero di minuti trascritti

3. Quanto si somigliano le due trascrizioni FS/WA? In termini di:
   1. il numero di unità di trascrizione
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   2. la durata media in ms delle unità di trascrizione
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   3. la durata media in ms delle unità di trascrizione
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   4. numero di span con overlap
   5. numero di span con pace variato
   6. numero di token con pattern intonativo
   7. numero di span con volume variato
   8. numero di token con allungamenti

4. Calcoliamo Delta-s(Gold, FS), Delta-w(Gold, WA) su:
   1. il numero di unità di trascrizione varia tra fase FS e fase WA?
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   2. la durata media in ms delle unità di trascrizione varia tra fase FS e fase WA?
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   3. la durata media in ms delle unità di trascrizione varia tra fase FS e fase WA?
      1. proviamo ad escludere le unità che contengono solo token metalinguistici
   4. numero di span con overlap
   5. numero di span con pace variato
   6. numero di token con pattern intonativo
   7. numero di span con volume variato
   8. numero di token con allungamenti

5. MISMATCH (per esperti e per non esperti) sul livello ortografico
   1. numero di token che mismatchano nelle due condizioni (FS e WA):
      1. li classifichiamo a mano (es. quante congiunzioni? quanti filler? quante shortpauses?...)
      2. valutazione "gravità" del mismatch (presenza/assenza di un token, stesso lemma, token diversi...)
      3. edit distance tra i token mismatchanti per valutare se ci sono typos (es. "parlare" vs "prlare")

6. MATCH
   1. calcolo accuracy (e per l'overlap che è di tipo relazionale UAS-LAS (Unlabeled attachment score e labeled attachment score)) per ogni feature jefferson: differenza tra accuratezza dell'annotatore FS rispetto al Gold e accuratezza dell'annotatore WA rispetto al gold.
   2. media delle accuratezze per file e media delle accuratezze per feature
   3. per i token che matchano a inizio e fine di unità di trascrizione, valutazione della precisione di inizio/fine unità di trascrizione (non si può fare in termini di accuracy perché sono numeri reali)
   4. Relazioni tra tipi di mismatch per le features. Ci sono features che sono più predittive di altre, sulla possibilità che esistano mismatch in altre colonne? Per esempio: avere una bassa accuratezza sui prolungamenti è predittivo rispetto ad avere una bassa accuratezza sul volume della voce?

7. valutazione precisione unità a prescindere dal contenuto
   1. Creazione grafo di corrispondenza delle unità attribuite ad ogni parlante per i vari annotatori.